In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

In [9]:
dataset = pd.read_csv('../Data/dataset.csv')

dataset.head(3)

,B0043GG,B0043HH,B0043II,Protein,Leukocyte,SG,Bilirubin,Acidity,Nitrite,Urobilinogen,...,Urin_color_Dk.Yellow,Urin_color_Lt. Yellow,Urin_color_Lt.Orange,Urin_color_Orange,Urin_color_Other,Urin_color_Red,Urin_color_Reddish,Urin_color_Straw,Urin_color_Yellow,Urin_color_yellow
0,9.3,251.5,4.3,1,119,1.020,0.1,5.0,103842,0.7,...,0,0,0,0,0,0,0,0,1,0
1,5.1,548.3,9.1,4,106,1.020,0.0,5.0,108098,0.7,...,0,0,0,0,0,0,0,0,1,0
2,12.2,1590.0,23.5,6,657,1.016,0.1,5.0,27515,0.7,...,0,0,0,0,0,0,0,0,1,0


In [10]:
dataset.corr()['Urobilinogen'].abs().sort_values(ascending=False)

Urobilinogen             1.000000
Urin_color_Dk.Yellow     0.663746
B0043HH                  0.368495
Leukocyte                0.274540
Urin_color_Yellow        0.222366
Occult_blood             0.131899
Bilirubin                0.120701
Glucose                  0.076340
Acidity                  0.070447
Nitrite                  0.049850
Ketone_body              0.037152
Urin_color_Orange        0.031757
B0043II                  0.031692
Protein                  0.024815
Urin_color_Lt. Yellow    0.024695
B0043GG                  0.018176
Urin_color_DK.Orange     0.016899
Urin_color_Red           0.006414
SG                       0.004718
Urin_color_DK.Yellow     0.003429
Urin_color_Lt.Orange     0.002812
Urin_color_yellow        0.002787
Urin_color_Brown         0.002541
Urin_color_Reddish       0.002326
Urin_color_Other         0.001903
Urin_color_Dk. Yellow    0.001701
Urin_color_Dk.Orange     0.001155
Urin_color_Dk.Red        0.000720
Urin_color_Amber         0.000648
Urin_color_Str

In [11]:
# Selecing columns

dataset = dataset[['Urin_color_Dk.Yellow', 'B0043HH', 'Leukocyte', 'Urin_color_Yellow',
                   'Occult_blood', 'Bilirubin', 'Urobilinogen']]

dataset.head(3)

,Urin_color_Dk.Yellow,B0043HH,Leukocyte,Urin_color_Yellow,Occult_blood,Bilirubin,Urobilinogen
0,0,251.5,119,1,3,0.1,0.7
1,0,548.3,106,1,2,0.0,0.7
2,0,1590.0,657,1,40,0.1,0.7


In [12]:
dataset.shape

(74337, 7)

In [13]:
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

X.shape, y.shape

((74337, 6), (74337,))

# Train, Test Split

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

X_train.shape, X_test.shape

((59469, 6), (14868, 6))

In [13]:
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2022)

# X_train.shape, X_val.shape

((47575, 3), (11894, 3))

# Data Scaling

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
# X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

In [18]:
X_train[:5, :]

array([[-0.08534238, -0.25766079, -0.53870365,  0.26455711, -0.68592602,
         0.19038558],
       [-0.08534238, -0.25494616, -0.48245593,  0.26455711, -0.55110695,
        -0.54462091],
       [-0.08534238, -0.25781403, -0.55628106,  0.26455711,  1.72284137,
         0.55788882],
       [-0.08534238, -0.25744186, -0.48245593,  0.26455711,  2.00596142,
        -0.54462091],
       [-0.08534238, -0.22661775, -0.40863081, -3.7799022 , -0.12867386,
         0.55788882]])

# Modeling

## 1. RandomForestRegressor

In [48]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold

In [49]:
params = {
    'n_estimators' : range(100, 200, 5),
    'verbose' : [True]
}

In [50]:
rfr = RandomForestRegressor(n_jobs=8, random_state=2022)

In [51]:
# 0값 확인
# 0값 존재 -> MAPE 사용 불가 (실제 y값으로 나누기 때문(y > 0 필수))

len(dataset.loc[dataset.Urobilinogen == 0])

2400

In [52]:
kfold = KFold(5, shuffle=True, random_state=2022)

### Learning

In [53]:
grid = GridSearchCV(rfr, params, n_jobs=8, cv=kfold, refit=True)
grid.fit(X_train, y_train)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 170 out of 170 | elapsed:    2.4s finished


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestRegressor(n_jobs=8, random_state=2022,
                                             verbose=True),
             n_jobs=8,
             param_grid={'n_estimators': range(100, 200, 5), 'verbose': [True]})

In [54]:
grid.best_params_

{'n_estimators': 170, 'verbose': True}

In [55]:
y_pred = grid.best_estimator_.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 170 out of 170 | elapsed:    0.0s finished


### Evaluating

In [56]:
from sklearn.metrics import mean_absolute_percentage_error, \
                            mean_absolute_error, mean_squared_error

In [57]:
mean_absolute_percentage_error(y_test, y_pred)

36396827841911.56

In [58]:
mean_absolute_error(y_test, y_pred)

0.15374442785604836

In [59]:
np.sqrt(mean_squared_error(y_test, y_pred))

0.5390370020841385

### Comparising y_pred, y_result

In [60]:
comparison = pd.DataFrame({'prediction' : y_pred, 'result' : y_test.values.ravel()})

In [61]:
comparison

,prediction,result
0,0.500000,0.5
1,0.300000,0.3
2,1.379322,0.8
3,0.500000,0.5
4,0.025882,0.0
...,...,...
14863,0.300000,0.3
14864,0.500000,0.5
14865,0.432941,0.6
14866,0.900000,0.9


### Saving Model

In [62]:
import pickle

In [67]:
saved_model = pickle.dump(grid.best_estimator_, open('../Model/{0}_RFT_n_{1}.pkl' \
    .format(dataset.columns[-1], grid.best_params_.get('n_estimators')), 'wb'),
                          protocol=pickle.HIGHEST_PROTOCOL)

### Loading Model

In [68]:
loaded_model = pickle.load(open('../Model/{0}_RFT_n_{1}.pkl' \
    .format(dataset.columns[-1], grid.best_params_.get('n_estimators')), 'rb'))

In [69]:
y_pred = loaded_model.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 170 out of 170 | elapsed:    0.0s finished


In [71]:
mean_absolute_error(y_test, y_pred)

0.15374442785604836

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))